# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:
!pip install kagglehub
!pip install azure-ai-ml azure-identity
!pip uninstall psutil -y
!pip install psutil==5.9.0

Found existing installation: psutil 7.0.0
Uninstalling psutil-7.0.0:
  Successfully uninstalled psutil-7.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 kB 12.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-nvdashboard 0.13.0 requires jupyterlab>=4, but you have jupyterlab 3.6.8 which is incompatible.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you have dask 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires distributed>=2024.4.1, but you have distributed 2023.2.0 which is incompatible.
azureml-training-tabular 1.60.0 requires scipy<1.11.0,>=1.0.0, but you have scipy 1.11.0 which is incompatible.
azureml-automl-dnn-nlp 1.60.0 requires torch==2.2.2, but you have torch 2.6.0 which is incompatible.


In [4]:
import os
import shutil

import kagglehub
import pandas as pd

from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Experiment
from azureml.core.authentication import AzureCliAuthentication

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
# Fetch data from Kaggle and move it to src folder of project
source_path = kagglehub.dataset_download("blastchar/telco-customer-churn")
print("Download Path to dataset files:", source_path)

shutil.move(source_path, os.getcwd()+"/Data")
print("New Path to dataset files:", os.getcwd()+"/Data")

# ML-CLient Credentials
subscription_id = "5a4ab2ba-6c51-4805-8155-58759ad589d8"
resource_group = "aml-quickstarts-282802"
workspace_name = "quick-starts-ws-282802"

# Get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)

my_data = Data(
    path=os.getcwd()+"/Data/WA_Fn-UseC_-Telco-Customer-Churn.csv",
    type=AssetTypes.URI_FILE,
    description="The IBM Telco customer churn data contains information about a fictional telco company that provided home phone and Internet services to 7043 customers in California in Q3",
    name="ibm-telco-churn-data"
)

ml_client.data.create_or_update(my_data)

print(f"Data asset '{my_data.name}' created successfully.")

Data asset 'ibm-telco-churn-data' created successfully.


In [7]:
data_asset = ml_client.data.get("ibm-telco-churn-data", version="2")
df = pd.read_csv(data_asset.path)
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
mask = df.columns != 'Churn'
train_features = df.loc[:, mask]
train_labels = df[['Churn']]

ws = Workspace(
            subscription_id=subscription_id,
            resource_group=resource_group,
            workspace_name=workspace_name,
            auth=AzureCliAuthentication()
        )

# # ws = Workspace.from_config()
# ws = ml_client.workspaces.get(name=ml_client.workspace_name)

# choose a name for experiment
experiment_name = 'churn-auto-ml'

experiment=Experiment(ws, experiment_name)

# # # TODO: Put your automl settings here
# # automl_settings = {}

# # TODO: Put your automl config here
automl_config = AutoMLConfig(task="classification",
                             X=train_features,
                             y=train_labels,
                             iterations=30,
                             iteration_timeout_minutes=5,
                             primary_metric="AUC_weighted",
                             n_cross_validations=5)

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
2025-06-10 17:02:48.418033: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749574970.000198   39255 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749574970.515281   39255 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749574974.850693   39255 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target mo

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
churn-auto-ml,AutoML_7ad23910-fe1d-47d3-a5b5-93f85a752db2,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected i

2025/06/10 17:03:46 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
